# Test Case LVV-T1600
This case will verify that the integration of the Camera Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s)powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running
* KAFKA producer running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

from lsst.ts.idl.enums.MTHexapod import ApplicationStatus


/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-sp.xml
summit


In [3]:
#This step should fail at NCSA since there is only one domain
print(os.environ["LSST_DDS_DOMAIN_ID"])

11


In [4]:
print(os.environ["LSST_DDS_INTERFACE"])

net1


The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [5]:
#Wait for history data [sec] default value is 60
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
#os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [6]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [7]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)


In [8]:
start_time = datetime.now()
test_message = "Camera Hexapod Integration Test"
#What means index=1 here?
#script = salobj.Controller("Script", index=1)
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

INFO:Script:Read historical data in 0.00 sec


time to start is 0:00:12.015711 [s]


In [9]:
await script.start_task

This is how you start the remote for the CSC.

In [11]:
# create the remote
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 1
hexapod_csc = salobj.Remote(name="MTHexapod", domain=script.domain, index=csc_index)

INFO:MTHexapod:Read historical data in 0.00 sec


In [12]:
await hexapod_csc.start_task

In [13]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

In [15]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

[<State.STANDBY: 5>, <State.OFFLINE: 4>]

In [66]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

RuntimeError: Error on cmd=cmd_standby, initial_state=3: msg='Command failed', ackcmd=(ackcmd private_seqNum=1873806286, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: You must use the clearError command or the engineering user interface to clear a rotator fault.')

In [19]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>]

In [20]:
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [21]:
print(hexapod_csc)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)
print(hexapod_csc.salinfo.index)

Hexapod hex 1
1


In [22]:
#This cell makes sure that the data go to EFD
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [23]:
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [24]:
dfe

,MTHexapodID,priority,private_host,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,summaryState
2021-03-08 18:33:54.509000+00:00,1,0,0,MTHexapod:1,1.615228e+09,5360,1.615228e+09,959a0a03,16,1.615228e+09,3
2021-03-08 18:45:47.504000+00:00,1,0,0,MTHexapod:1,1.615229e+09,5360,1.615229e+09,959a0a03,17,1.615229e+09,5
2021-03-08 18:46:11.263000+00:00,1,0,0,MTHexapod:1,1.615229e+09,5360,1.615229e+09,959a0a03,18,1.615229e+09,4
2021-03-08 18:47:44.543000+00:00,1,0,0,MTHexapod:1,1.615229e+09,5360,1.615229e+09,959a0a03,19,1.615229e+09,5
2021-03-08 18:47:55.433000+00:00,1,0,0,MTHexapod:1,1.615229e+09,5360,1.615229e+09,959a0a03,20,1.615229e+09,1
2021-03-08 18:48:04.571000+00:00,1,0,0,MTHexapod:1,1.615229e+09,5360,1.615229e+09,959a0a03,21,1.615229e+09,2
2021-03-08 19:00:29.915000+00:00,1,0,0,MTHexapod:1,1.615230e+09,5360,1.615230e+09,959a0a03,22,1.615230e+09,1
2021-03-08 19:00:29.965000+00:00,1,0,0,MTHexapod:1,1.615230e+09,5360,1.615230e+09,959a0a03,23,1.615230e+09,5
2021-03-08 19:00:36.040000+00:00,1,0,0,MTHexapod:1,1.615230e+09,5360,1.615230e+09,959a0a03,24,1.615230e+09,4
2021-03-08 19:00:38.241000+00:00,1,0,0,MTHexapod:1,1.615230e+09,5360,1.615230e+09,959a0a03,25,1.615230e+09,5


In [25]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [26]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [27]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.") 

In [28]:
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
#This is only for the real system. The simulater only has the AVAILABLE offline state 

'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

2


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await hexapod_csc.cmd_enable.set_start()

In [65]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=155001503, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Failed: final state is <State.FAULT: 3> instead of <State.STANDBY: 5>')

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
#hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
#await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730, u=0.17,v=0,w=0,sync=True)



In [67]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_stop.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2108132280, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Not enabled')

EnabledSubstate.STATIONARY
EnabledSubstate.STATIONARY


In [54]:
#This command is to set the Hexapod to zero position
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


In [31]:
script.log.info(f"TEST to see if annotations like this arrive in the EFD.")

INFO:Script:TEST to see if annotations like this arrive in the EFD.


In [32]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 7 - Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7 - Starting time: 2021-03-08 19:05:44.555162 UTC


hex position
     -0.18       -0.93       -0.01       -0.00       -0.00        0.00  



In [33]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 7")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7


hex position
    500.17     -501.01      200.03        0.01       -0.02       -0.00  



In [34]:
#This command is to set the Hexapod to zero position
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

EnabledSubstate.MOVING_POINT_TO_POINT


In [35]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

EnabledSubstate.STATIONARY


# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [36]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [41]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 11- Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 11- Starting time: 2021-03-08 19:17:38.449505 UTC


hex position
     -0.54       -0.73       -0.11       -0.00        0.00       -0.00  



In [42]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


#cs = await hexapod_csc.evt_controllerState.aget()
#print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
#if cs.enabledSubstate != MTHexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {MTHexapod.EnabledSubstate(cs.enabledSubstate)}")

# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != MTHexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.STATIONARY}. It is {MtHexapod.EnabledSubstate(cs.enabledSubstate)}")

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 11")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 11


hex position
      0.51        0.45     1516.90        0.00       -0.00        0.00  



In [43]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [63]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


In [45]:
await asyncio.sleep(STD_WAIT)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


This blocks move the hexapod via the lookup table:

CAVE: The setCompensationMode expects infrormation from the MTmount(elevation) and MTRotator(rotation).

To use the setCompensationMode at the summit the controller for MTMount and MTRotator need to run.
To test this at NCSA the simulators must be stopped first.

Use the notebooks MTMount-Controller and MTRotator-Controller to stop the simulators, start the Controller and reverse the steps after finishing testing.

In [ ]:
rot = salobj.Remote(script.domain, "MTRotator")

In [ ]:
await asyncio.gather(rot.start_task,
                     hex.start_task)

In [ ]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 17- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
#Switch CompensationMode on
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

In [ ]:
#move the hexapod Not allowed here! First leave compensation mode than move to new postion!!! Otherwise the you hexpod gets stuck in compensation  mode
#await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

#await asyncio.sleep(10.)

In [ ]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Switch CompensationMode off 
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 17")

In [ ]:
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)


In [ ]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
posC = await hexapod_csc.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

posU = await hexapod_csc.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posU, i), end= ' ')
print('\n')

Analysis of the LUT correction

In [ ]:
LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
print(LUTfile)
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexapod_csc.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexapod_csc.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']

In [ ]:
pred = []
elev=89.999
print('Predicted LUT compensation:')
for i in range(6):
    coeff = elevCoeff[i] #starts with C0
    mypoly = np.poly1d(coeff[::-1]) #poly1d needs C5 first
    pred.append(mypoly(elev))
    print('%10.2f '%pred[i], end= ' ')
print('\n')
print('Actual LUT compensation:')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [46]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 24 - Starting time: {now} UTC")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 24 - Starting time: 2021-03-08 19:38:13.266269 UTC


hex position
     -0.24       -0.69       -0.08       -0.00        0.00       -0.00  



In [47]:
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
    500.03      800.46      200.15        0.00        0.00        0.00  



In [48]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 24")


# cs = await MThexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await MThexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.STATIONARY}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await MThexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 24


hex position
    500.38      799.75      700.30       -0.00       -0.00        0.00  



In [49]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [55]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29 - Starting time: {now} UTC")#Pivot test
#Step 29
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29 - Starting time: 2021-03-08 19:51:52.624748 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   2000.24    -3500.45      200.09        0.01       -0.05        0.00  



In [56]:
#Step 30 -- set a new Pivot point
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(10.)

In [60]:
#Step 31
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   2000.29    -3500.03      200.20        0.01       -0.05        0.00  



In [62]:
#Set the pivot point back to the original value (0.703m in um)
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=1938000)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 29


hex position
   2000.23    -3500.04      200.28        0.01       -0.05        0.00  



# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [64]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test - Starting time: {now} UTC")
for i in range(100):
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-5660,y=0,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0,v=-0.17,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-5660,z=-7730,u=-0.17,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
now = datetime.now()
script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test - Stopping time: {now} UTC")

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Endurance Test - Starting time: 2021-03-08 20:08:02.201129 UTC


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.MOVING_POINT_TO_POINT


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1015964321, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.FAULT: 3> instead of <State.ENABLED: 2>')

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection